In [ ]:
from tsi_toolkit import *
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [ ]:
def simulate_time_series(
    n_points,  # Total duration of the time series (in arbitrary units, e.g., seconds or days)
    sampling_interval,  # Sampling interval for the time series
    alpha=2.0,  # Slope of the red noise PSD (alpha = 1 corresponds to 1/f noise)
    random_seed=None  # Optional seed for reproducibility
):
    if random_seed is not None:
        np.random.seed(random_seed)

    time = np.linspace(0, n_points*sampling_interval, n_points)
    dt = time[1] - time[0]
    freqs = np.fft.rfftfreq(n_points, d=dt)
    
    # Define the red noise PSD: P(f) propto 1 / f^alpha
    psd = 1 / (freqs + 1e-5) ** alpha  # Avoid division by zero
    
    random_phases = np.exp(2j * np.pi * np.random.rand(len(freqs)))
    amplitudes = np.sqrt(psd)
    fourier_coeffs = amplitudes * random_phases
    
    red_noise = np.fft.irfft(fourier_coeffs, n=n_points)

    red_noise = 10 * red_noise / np.std(red_noise)
    red_noise = red_noise - np.mean(red_noise) + 100

    # introduce white noise
    #counts = red_noise * dt
    #flux = np.random.normal(loc=counts, scale=np.sqrt(counts))
    #flux = flux / dt
    #fluxsigmas = np.sqrt(flux)
    flux = red_noise
    fft = np.fft.rfft(flux)
    power_spectrum = np.abs(fft) ** 2

    freq_mask = (freqs >= 1e-8) & (freqs <= 1/(2 * dt))
    freqs = freqs[freq_mask]
    power_spectrum = power_spectrum[freq_mask]
    fft = fft[freq_mask]

    fluxsigmas = None
    
    return time, flux, fluxsigmas, freqs, fft, power_spectrum

# Example usage
n_points = 1000
sampling_interval = 1
time, time_series, time_series_sigmas, freqs, fft, power_spectrum = simulate_time_series(
    n_points=n_points, 
    sampling_interval=sampling_interval, 
    alpha=2, 
    random_seed=40
)

time2, time_series2, time_series_sigmas2, freqs2, fft2, power_spectrum2 = simulate_time_series(
    n_points=n_points,
    sampling_interval=sampling_interval, 
    alpha=1, 
    random_seed=43
)

timeseries_object1 = TimeSeries(time, time_series)
timeseries_object1.plot()
timeseries_object2 = TimeSeries(time2, time_series2)

In [ ]:
### test array imports
# verify psd
power_spectrum_tsi = PowerSpectrum(times=time, values=time_series, norm=False)

# Check if power_spectrum_tsi.powers agrees with power_spectrum
assert np.allclose(power_spectrum_tsi.freqs, freqs), "The frequencies do not match!"
assert np.allclose(power_spectrum_tsi.powers, power_spectrum), "The power spectra do not match!"

# verify cross spectrum
true_cross_spectrum = np.conj(fft) * fft2
cross_spectrum_tsi = CrossSpectrum(times1=time, values1=time_series, times2=time2, values2=time_series2, norm=False)

# Check if cross_spectrum_tsi.cs agrees with true_cross_spectrum
assert np.allclose(cross_spectrum_tsi.cs, true_cross_spectrum), "The cross spectra do not match!"
# verify lag frequency spectrum
true_lag_spectrum = np.angle(true_cross_spectrum) / (2 * np.pi * freqs)
lag_spectrum_tsi = LagFrequencySpectrum(
    times1=time, values1=time_series, times2=time2, values2=time_series2, subtract_coh_bias=False
    )

# Check if lag_spectrum_tsi.lags agrees with true_lag_spectrum
assert np.allclose(lag_spectrum_tsi.lags, true_lag_spectrum), "The lag spectra do not match!"



### test object imports
# verify psd
power_spectrum_tsi = PowerSpectrum(timeseries=timeseries_object1, norm=False)

# Check if power_spectrum_tsi.powers agrees with power_spectrum
assert np.allclose(power_spectrum_tsi.freqs, freqs), "The frequencies do not match!"
assert np.allclose(power_spectrum_tsi.powers, power_spectrum), "The power spectra do not match!"

# verify cross spectrum
true_cross_spectrum = np.conj(fft) * fft2
cross_spectrum_tsi = CrossSpectrum(timeseries1=timeseries_object1, timeseries2=timeseries_object2, norm=False)

# Check if cross_spectrum_tsi.cs agrees with true_cross_spectrum
assert np.allclose(cross_spectrum_tsi.cs, true_cross_spectrum), "The cross spectra do not match!"
# verify lag frequency spectrum
true_lag_spectrum = np.angle(true_cross_spectrum) / (2 * np.pi * freqs)
lag_spectrum_tsi = LagFrequencySpectrum(
    timeseries1=timeseries_object1, timeseries2=timeseries_object2, subtract_coh_bias=False
    )

# Check if lag_spectrum_tsi.lags agrees with true_lag_spectrum
assert np.allclose(lag_spectrum_tsi.lags, true_lag_spectrum), "The lag spectra do not match!"

In [ ]:
# test GP noise homoskedastic noise
timeseries_object1 = TimeSeries(time, time_series)
power_spectrum_tsi = PowerSpectrum(timeseries=timeseries_object1, norm=True)
plt.scatter(power_spectrum_tsi.freqs, power_spectrum_tsi.powers, s=2)
plt.xscale('log')
plt.yscale('log')
plt.show()

# time_series_adj = time_series / np.std(time_series)
# time_series_adj = time_series_adj - np.mean(time_series_adj)
# print(np.mean(np.abs(time_series_adj)))
# timeseries_object1 = TimeSeries(time, time_series_adj)
# power_spectrum_tsi = PowerSpectrum(timeseries=timeseries_object1, norm=False)
# plt.scatter(power_spectrum_tsi.freqs, power_spectrum_tsi.powers, s=2)
# plt.xscale('log')
# plt.yscale('log')
# plt.show()

# mean_sigma1 = np.mean(np.sqrt(time_series_adj))
# mean1 = np.mean(time_series_adj)
# nyquist_freq = 1 / (2 * (time[1] - time[0]))
# pnoise1 = mean_sigma1 ** 2 / ( nyquist_freq * mean1 ** 2 )
# print(pnoise1)

In [ ]:
from pylag import GPLightCurve
gp_pylag = GPLightCurve(t=time, r=time_series,
                        kernel='matern12',
                        lognorm=False, noise_kernel=True, 
                        run_fit=True
                              )

In [ ]:
param_values = gp_pylag.get_fit_param(log_par=False)
params = gp_pylag.make_param_dict(param_values, log_par=False)
print(params)


In [ ]:
timeseries_object1 = TimeSeries(time, time_series)
gp_model = GaussianProcess(timeseries=timeseries_object1,
                           kernel_form='Matern12', white_noise=True, 
                           train_iter=1000, learn_rate=1e-0,
                             verbose=True)

plt.scatter(gp_model.train_times, gp_model.train_values, s=2)
plt.show()

In [ ]:
gp_model.get_hyperparameters()

In [ ]:
import gpytorch
import torch
def bic(lml, num_params, num_data):
    return -2 * lml + num_params * np.log(num_data)

def train_gp_model(train_x, train_y, kernel, lr = 0.01, training_iter = 2000, verbal = False):
    
    def create_gp_model(train_x, train_y, likelihood, kernel):
        class GPModel(gpytorch.models.ExactGP):
            def __init__(self, train_x, train_y, likelihood):
                super(GPModel, self).__init__(train_x, train_y, likelihood)
                self.mean_module = gpytorch.means.ZeroMean()
                if kernel == 'Matern12':
                    self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=0.5))
                elif kernel == 'Matern32':
                    self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=1.5))
                elif kernel == 'Matern52':
                    self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=2.5))
                elif kernel == 'RQ':
                    self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RQKernel())
                elif kernel == 'RBF':
                    self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
                else:
                    raise(ValueError('Invalid kernel type'))

            def forward(self, x):
                mean_x = self.mean_module(x)
                covar_x = self.covar_module(x)
                return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

        return GPModel(train_x, train_y, likelihood)

    # initialize likelihood and model
    likelihood = gpytorch.likelihoods.GaussianLikelihood(
        learn_additional_noise=True,
        noise_prior=gpytorch.priors.NormalPrior(0.1, 0.5), 
    )

    # create gp model
    model = create_gp_model(train_x, train_y, likelihood, kernel)

    # Find optimal model hyperparameters
    model.train()
    likelihood.train()

    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)  # Includes GaussianLikelihood parameters

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    for i in range(training_iter):
        # Zero gradients from previous iteration
        optimizer.zero_grad()

        # Output from model
        output = model(train_x)

        # Calc loss and backprop gradients
        loss = -mll(output, train_y)
        loss.backward()

        if verbal:
                print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
                    i + 1, training_iter, loss.item(),
                    model.covar_module.base_kernel.lengthscale.item(),
                    model.likelihood.noise.item()
                ))

        optimizer.step()

    num_params = sum([p.numel() for p in model.parameters()])
    inf_crit = bic(-loss.item(), num_params, len(train_x))
    
    return model, likelihood, inf_crit

In [ ]:
import torch

time_series_standardized = (time_series - np.mean(time_series)) / np.std(time_series)
plt.scatter(time, time_series_standardized, s=2)
plt.show()

train_x = torch.tensor(time).float()
train_y = torch.tensor(time_series_standardized).float()

# kernel comparison using bayesian information criterion
model, likelihood, inf_crit = train_gp_model(train_x, train_y, 
                                             'Matern12', lr = 1e1,
                                             verbal = True)

print('Model hypers: ', model.covar_module.base_kernel.lengthscale.item(), model.likelihood.noise.item())

# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()